# GPT-2 for Sentiment Analysis on IMDb movie reviews

## Table of Contents
1. [Introduction](##Introduction)
2. [Data exploration](##Data-Exploration)
3. [Zero Shot Classification](##Zero-shot-classification)

## Introduction

The [IMDb](https://ai.stanford.edu/~amaas/data/sentiment/) is a binary sentiment classification dataset consisting of 100k movie reviews(50k positive and 50k negative). The dataset is split into train and test containing 50k reviews each.

In this notebook, my goals are:
1. Understand and implement [GPT-2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf). Run GPT-2 on the IMDb classification task.
2. Fine-tune GPT-2 for sentiment classification in under ~30 minutes on a 8GB Apple M2 macbook air (Faster if you have a Nvidia GPU).
3. Understand how [LoRA](https://arxiv.org/abs/2106.09685) is implemented and use it to fine-tune GPT-2 for sentiment classification.

## Data-Exploration
Get a summary of the dataset. i.e
1. No of samples
2. No of positive / negative samples.
3. Length of the movie reviews



In [1]:
import pandas
import torch
from torch.utils.data import Dataset

from gpt_config import GPTConfig
from sentiment_classification.reviewsDataset import reviewsDataset
from sentiment_classification.eval import Eval
from sentiment_classification.eval_config import EvalConfig
from sentiment_classification.train import Trainer
from sentiment_classification.train_config import TrainConfig


In [ ]:
# Dataset exploration

imdb_train = reviewsDataset("train",max_length=10000)
imdb_test = reviewsDataset("test",max_length=10000)


def format_data(dataset: Dataset) -> pandas.DataFrame:

    data = []
    for batch in dataset:
        data.append({"input_ids":len(batch["input_ids"]),
                    "label": batch["label"],
                    "filename": batch["fpath"]})
    
    return pandas.DataFrame(data)

train_data = format_data(imdb_train)
test_data = format_data(imdb_test)


*Summary statistics of the dataset*

In [ ]:
def summary(data: pandas.DataFrame) -> None:
    print(f"Number of reviews: {len(data)}")
    print(f"Positive Reviews: {data[data['label'] == 1]['label'].count()}")
    print(f"Negative Reviews: {data[data['label'] == 0]['label'].count()}")
    print(f"Max Review Length: {data['input_ids'].max()}\nMin Review Length: {data['input_ids'].min()}")
    print(f"Median Review Length: {data['input_ids'].median()}\nMean Review Length: {data['input_ids'].mean()}")

print("Train\n--------------")
summary(train_data)
print("Test\n---------------")
summary(test_data)

*Length of reviews (measured by the number of tokens)*

In [ ]:
from matplotlib import pyplot as plt
def plot_hist(title: str,df: pandas.DataFrame) -> None:
    plt.figure()
    plt.hist(df["input_ids"],bins=100)
    plt.xlabel(f"No of tokens")
    plt.ylabel("Count")
    plt.title(f"{title}")

plot_hist(title='Train Data', df=train_data) 
plot_hist(title="Test Data", df=test_data)   

In [ ]:
plot_hist(title="Positive Reviews Test",df=test_data[test_data['label']==1])
plot_hist(title="Negative Reviews Test",df=test_data[test_data['label']==0])

Run the test.py in `sentiment_classification` and write the results to a file

In [2]:
def get_metrics_by_bin(results, bins,threshold=0.5):
    TP = len(results[(results["label"] >= threshold) & (results["prediction"] >= threshold)])
    FP = len(results[(results["label"] < threshold) & (results["prediction"] >= threshold)])
    TN = len(results[(results["label"] < threshold) & (results["prediction"] < threshold)])
    FN = len(results[(results["label"] > threshold) & (results["prediction"] < threshold)])
    
    print("Metrics")
    print(f"Precision: {TP/(TP+FP)}\nRecall: {TP/(TP+FN)}\nAccuracy: {(TP+TN)/len(results)}")
    bins = range(0,1500,128)
    results["bin"] = pandas.cut(results['length'],bins)
    metrics_by_bin = results.groupby('bin').apply(lambda x: pandas.Series({"TP": ((x["label"] >= threshold) & (x["prediction"] >= threshold)).sum(),
                                                                            "FP":((x["label"] < threshold) & (x["prediction"] >= threshold)).sum(),
                                                                            "FN": ((x["label"] >= threshold) & (x["prediction"] < threshold)).sum(),
                                                                            "TN": ((x["label"] < threshold) & (x["prediction"] < threshold)).sum()}))

    metrics_by_bin["accuracy"] = (metrics_by_bin["TP"] + metrics_by_bin["TN"])/(metrics_by_bin["TP"] + metrics_by_bin["TN"]+ metrics_by_bin["FP"]+ metrics_by_bin["FN"])
    metrics_by_bin["precision"] = metrics_by_bin["TP"]/(metrics_by_bin["TP"] + metrics_by_bin["FP"])
    metrics_by_bin["recall"] = metrics_by_bin["TP"]/(metrics_by_bin["TP"] + metrics_by_bin["FN"])
    print("Metrics by bin")
    print(metrics_by_bin.to_markdown())

Predict the next word given the following prompt
 
'''
Review: The movie was awesome. Sentiment: Positive. 
Review: The performances were disappointing. Sentiment: Negative. 
Review: {review} Sentiment:
'''
I calculate the probabilities of the word " Positive" and " Negative" and classify the review based on which probability is greater.

**Run evaluation for the zero shot approach**

In [ ]:
model_config = GPTConfig(block_size=128,use_lora=False)
eval_config = EvalConfig(results_path="zero_shot_128.txt",subset=False)
test_set = reviewsDataset(split="test")
evaluator = Eval(test_set=test_set,eval_config=eval_config,model_config=model_config)
evaluator.evaluate()

In [ ]:
res_file = pandas.read_csv("zero_shot_128.txt")
bins = range(0,1500,128)
get_metrics_by_bin(res_file,bins,threshold=0.5)

**Finetuning without LoRA**

In [ ]:
train_config = TrainConfig(out_dir="run/first_part_review_no_lora/",init_from="gpt2",checkpoint_name="finetune_no_lora.ckpt")
model_config = GPTConfig(use_lora=False)
rd = reviewsDataset(split="train",max_length=model_config.block_size)
train_set, val_set = torch.utils.data.random_split(rd,[0.85,0.15])
trainer = Trainer(train_set,val_set,train_config,model_config)
trainer.train()

**Run eval using the fine-tuned model**

In [ ]:
model_config = GPTConfig(block_size=128,use_lora=False,load_from_checkpoint=True,checkpoint_path="run/finetune_no_lora.ckpt")
eval_config = EvalConfig(results_path="finetuned_no_lora.txt",subset=False)
test_set = reviewsDataset(split="test")
evaluator = Eval(test_set=test_set,eval_config=eval_config,model_config=model_config)
evaluator.evaluate()

**Test the performance of the fine-tuned model**

In [ ]:
res_file = pandas.read_csv("finetuned_no_lora.txt")
bins = range(0,1500,128)
get_metrics_by_bin(res_file,bins,threshold=0.5)

**Run training using LoRA**

In [4]:
train_config = TrainConfig(out_dir="run/first_part_review",checkpoint_name="finetune_lora.ckpt",init_from="resume",learning_rate=5e-3)
model_config = GPTConfig(block_size=128,use_lora=True,r=8,binary_classification_head=True)
rd = reviewsDataset(split="train",max_length=model_config.block_size)
train_set, val_set = torch.utils.data.random_split(rd,[0.85,0.15])
trainer = Trainer(train_set,val_set,train_config,model_config)
trainer.train()

Loading pre-trained weights for gpt2
Number of parameters: 123.65M
Resuming training from run/first_part_review/finetune_lora.ckpt


/Users/varun/Documents/learning/Projects/ml-practice/transformers/sentiment_classification/train.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.ckpt = torch.load(

num decayed parameter tensors: 25, with 295,680 parameters
num non-decayed parameter tensors: 0, with 0 parameters


  0%|          | 0/52000 [00:00<?, ?it/s]

Step: 8000
 Train Loss: 0.5172512531280518
Validation Loss: 0.5576334595680237
Saving checkpoint to run/first_part_review/finetune_lora.ckpt


...

 92%|█████████▏| 48000/52000 [2:52:12<14:00,  4.76it/s]  

Step: 56000
 Train Loss: 0.20802640914916992
Validation Loss: 0.4687481224536896
Saving checkpoint to run/first_part_review/finetune_lora.ckpt


 96%|█████████▌| 50000/52000 [2:59:23<07:09,  4.66it/s]  

Step: 58000
 Train Loss: 0.24838979542255402
Validation Loss: 0.44210702180862427
Saving checkpoint to run/first_part_review/finetune_lora.ckpt


100%|██████████| 52000/52000 [3:06:39<00:00,  4.64it/s]  


**Evaluate using the LoRA finetuned model**

In [7]:
model_config = GPTConfig(use_lora=True,binary_classification_head=True,block_size=128,load_from_checkpoint=True,checkpoint_path="run/first_part_review/finetune_lora.ckpt")
eval_config = EvalConfig(results_path="finetuned_lora.txt",batch_size=2,subset=True)
test_set = reviewsDataset(split="test")
evaluator = Eval(test_set=test_set,eval_config=eval_config,model_config=model_config)
evaluator.evaluate()

Loading pre-trained weights for gpt2
Number of parameters: 123.65M


/Users/varun/Documents/learning/Projects/ml-practice/transformers/sentiment_classification/eval.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.m

**Test the performance of the LoRA finetuned model**

In [8]:
res_file = pandas.read_csv("finetuned_lora.txt")
bins = range(0,1500,128)
get_metrics_by_bin(res_file,bins,threshold=0.5)

Metrics
Precision: 0.7698412698412699
Recall: 0.776
Accuracy: 0.772
Metrics by bin
| bin          |   TP |   FP |   FN |   TN |   accuracy |   precision |     recall |
|:-------------|-----:|-----:|-----:|-----:|-----------:|------------:|-----------:|
| (0, 128]     |   20 |    5 |    3 |   10 |   0.789474 |    0.8      |   0.869565 |
| (128, 256]   |   47 |   13 |   11 |   54 |   0.808    |    0.783333 |   0.810345 |
| (256, 384]   |   16 |    4 |    5 |   15 |   0.775    |    0.8      |   0.761905 |
| (384, 512]   |    5 |    5 |    4 |   12 |   0.653846 |    0.5      |   0.555556 |
| (512, 640]   |    7 |    0 |    3 |    2 |   0.75     |    1        |   0.7      |
| (640, 768]   |    1 |    1 |    1 |    2 |   0.6      |    0.5      |   0.5      |
| (768, 896]   |    0 |    1 |    1 |    1 |   0.333333 |    0        |   0        |
| (896, 1024]  |    1 |    0 |    0 |    0 |   1        |    1        |   1        |
| (1024, 1152] |    0 |    0 |    0 |    0 | nan        |  nan     

/var/folders/p1/qmcgqwhj1mx925j0j_r36xp40000gn/T/ipykernel_32570/3153587618.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  metrics_by_bin = results.groupby('bin').apply(lambda x: pandas.Series({"TP": ((x["label"] >= threshold) & (x["prediction"] >= threshold)).sum(),
/var/folders/p1/qmcgqwhj1mx925j0j_r36xp40000gn/T/ipykernel_32570/3153587618.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics_by_bin = results.groupby('bin').apply(lambda x: pandas.Series({"TP": ((x["label"] >= threshold) & (x["